In [1]:
import pandas as pd
import numpy as np

import datetime

from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas()

import xgboost as xgb 

from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit, train_test_split
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.metrics import mean_squared_error

import graphviz

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def fillnan(data, years_c):
    for name in years_c:
        #заменяем троеточия
        data[name] = data[name].apply(lambda x: np.nan if x==('..') else float(x))

    return data

def dropempt(data, years_c):
    data = fillnan(data, years_c)
    return data.dropna(thresh=10)

def Clearing(data):
    years_c = [item for ind, item in enumerate(np.array(data.columns)) if item not in ['Country Name', 'Country Code','Series Code','Series Name']]
    print ('Before clear:', len(data))

    data = dropempt(data, years_c)

    print ('After clear:', len(data))
    return data

In [3]:
def Make_region(code, reg='region', by = 'name'):
    #print code
    if code in data_cnt[by].values:
        #print code
        name = (data_cnt[by][data_cnt[by] == code].index.tolist())[0]
        return data_cnt[reg].at[name]
    else:
        return np.nan

def Sorting(data, reg='region'):
    #if data['Country Name'].get_value(len(data)-5) == np.nan:
    #    data=data.drop(data.index[[range(len(data)-5, len(data))]]).reset_index()
    if 'Region' not in data.columns:
        data['Region'] = data['Country Name'].progress_apply(lambda x: Make_region(x, reg))
        data['Region'].dropna()
        return data.reset_index()
    else:
        return data

In [4]:
df = pd.read_excel('data/Data_Extract_From_Gender_Statistics.xlsx', encoding = 'utf8').\
            append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', encoding='utf8')).\
            append(pd.read_excel('data/Data_Extract_From_Millennium_Development_Goals.xlsx', encoding='utf8')).\
            append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics_by_Wealth_Quintile.xlsx',encoding='utf8'))

data_cnt = pd.read_csv('all.csv', encoding='utf8')
df = Clearing(df)
defen = pd.read_excel('data/Data_Extract_From_Gender_Statistics.xlsx', sheet_name=1, encoding = 'utf8').\
               append(pd.read_excel('data/Data_Extract_From_Health_Nutrition_and_Population_Statistics.xlsx', sheet_name=1, encoding='utf8')).\
               append(pd.read_excel('data/Data_Extract_From_Millennium_Development_Goals.xlsx', sheet_name=1, encoding='utf8'))

df = Sorting(df, 'sub-region')
df.drop_duplicates(inplace = True)
df.drop(columns = ['index','2016 [YR2016]'], inplace=True)

Before clear: 398541
After clear: 104445


In [7]:
def Testing_grid_xgb(X_train, Y_train):
    cv = TimeSeriesSplit(n_splits=8)
    clf = xgb.XGBRegressor()        
    parameters = {
        'scale_pos_weight' : [1, 1.5, 1.2],
        'objective':['reg:linear'],
        'learning_rate': [0.05, 0.1, 0.2, 0.3],
        'max_depth': [3 ,4 ,6, 5],
        'min_child_weight': [2, 3 ,4, 5],
        'max_delta_step': [0, 0.2, 0.8],
        'reg_lambda': [0.5, 0.8, 0],
        'reg_alpha': [0.5, 0.8, 0],
        'silent': [1],
        'subsample': [0.2, 0.05, 0.3, 0.4],
        'colsample_bytree': [0.7, 0.8, 1],
        'n_estimators': [100 ,250, 450],
        'base_score' : [0.5, 1],
        'gamma': [0, 0.5, 1]
    }
    
    start_time = datetime.datetime.now()
    gs = RandomizedSearchCV(clf, scoring='neg_mean_squared_error', param_distributions=parameters, 
                      cv=cv, n_jobs = -1, verbose=True,  n_iter=10000)
    gs.fit(X_train,Y_train)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    
    means = gs.cv_results_['mean_test_score']
    
    print (max(means))
    return gs.best_params_

In [8]:
def makexy(df, country = 'RUS'):
    #Years
    years_c = [item for ind, item in enumerate(np.array(df.columns)) if item not in ['Region','Country Name', 'Country Code','Series Code','Series Name', 'index']]
    
    #split country
    df = df[(df['Country Code']== country)].copy()
    df.drop_duplicates(inplace = True)
    
    y = df[df['Series Name'] == 'Life expectancy at birth, total (years)'][years_c].interpolate(axis=1).T
    
    if y.shape[1] == 1:
        y.columns = ['Life expectancy at birth, total (years)']
    else:
        return pd.DataFrame(), pd.DataFrame()
    
    #Transponenting and then adding 
    mask = ((df['Series Name'] != 'Life expectancy at birth, male (years)') & 
            (df['Series Name'] != 'Life expectancy at birth, female (years)') &
            (df['Series Name'] != 'Life expectancy at birth, total (years)'))
    
    X = df[mask][years_c].T
    X.columns = df[(df['Country Code']== country) & mask]['Series Name']
    X.reset_index(inplace = True, drop = True)
    del df
    
    X['Years'] = [int(name.split()[0]) for name in years_c]
    X['Region_code'] = int(Make_region(country,'region-code', by = 'alpha-3'))
    X['Reion_sub_code'] = int(Make_region(country, 'sub-region-code', 'alpha-3'))
    X['Country_code'] = int(Make_region(country, 'country-code', 'alpha-3'))
    return X.reset_index(drop = True).fillna(method='pad'), y.reset_index(drop = True).fillna(method='pad')

def train_set(df, region = 'Eastern Europe'):
    x_train, y_train = pd.DataFrame(), pd.DataFrame()
    for country in df[df['Region'] == region]['Country Code'].unique():
        tmp = makexy(df, country)
        x_train = pd.concat([tmp[0], x_train], axis=0).fillna(-999)
        y_train = pd.concat([tmp[1], y_train], axis=0).fillna(-999)
    return x_train.as_matrix(), y_train.as_matrix(), x_train.columns, x_train
    

In [7]:
def Resmaking(df):
    for region in tqdm_notebook(df['Region'].dropna().unique()):
        tmp = train_set(df, region)
        X_train, X_test, y_train, y_test = train_test_split(tmp[0], tmp[1], test_size=0.25,
                                                            random_state=42)
        p = Testing_grid_xgb(X_train, y_train)
        
        model = xgb.XGBRegressor(max_depth=p['max_depth'], learning_rate = p['learning_rate'], 
                                 n_estimators = p['n_estimators'], objective = p['objective'], 
                                 gamma = p['gamma'], min_child_weight = p['min_child_weight'], 
                                 max_delta_step = p['max_delta_step'], subsample = p['subsample'], 
                                 colsample_bytree = p['colsample_bytree'], reg_alpha = p['reg_alpha'], 
                                 reg_lambda = p['reg_lambda'], scale_pos_weight = p['scale_pos_weight'], 
                                 base_score = p['base_score'])
        model.fit(X_train, y_train)
        
        print ('MSE : {0:f}%, \nIn {1:s}'.format(100-mean_squared_error(model.predict(X_test), y_test), region))
        
        model.fit(tmp[0], tmp[1])
        
        rez = pd.DataFrame(model.feature_importances_, index = tmp[2]).sort_values(by = [0], ascending=False)
        rez[rez[0]!=0].to_excel('Importance/Varible_reg_in_'+str(region)+'.xlsx')
        
        rez = pd.concat([pd.DataFrame(f_regression(tmp[0], tmp[1].ravel())[0], columns=['F-value'], index=tmp[2]),
                         pd.DataFrame(f_regression(tmp[0], tmp[1].ravel())[1], columns=['p-value'], index=tmp[2])], axis=1)
        rez['p-value'] = rez['p-value'].apply(lambda x: round(x, 5))
        
        rez[rez['p-value']<0.01].sort_values(by = ['F-value'], ascending=False).to_excel('Importance/Varible_f_reg_in_'+str(region)+'.xlsx')
        
        rez = pd.DataFrame(mutual_info_regression(tmp[0], tmp[1].ravel()),
                           index = tmp[2]).sort_values(by = [0], ascending=False)
        rez[rez[0]!=0].to_excel('Importance/Varible_muatal_reg_in_'+str(region)+'.xlsx')

In [8]:
Resmaking(df)

Fitting 8 folds for each of 10000 candidates, totalling 80000 fits


[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 691 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 1191 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 1891 tasks      | elapsed:   28.3s
[Parallel(n_jobs=-1)]: Done 2791 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 3891 tasks      | elapsed:   56.0s
[Parallel(n_jobs=-1)]: Done 5191 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 6691 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 8391 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 10291 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 12391 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 14691 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 17191 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 19891 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 22791 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 25891 tasks      | elap

Time elapsed: 0:17:33.847840
-1.827964772277412
MSE : 99.643631%, 
In Southern Asia


/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:298: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:303: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Fitting 8 folds for each of 10000 candidates, totalling 80000 fits


...

/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:298: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:303: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Fitting 8 folds for each of 10000 candidates, totalling 80000 fits


[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 1632 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 3032 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 4832 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done 7032 tasks      | elapsed:   32.9s
[Parallel(n_jobs=-1)]: Done 9632 tasks      | elapsed:   45.3s
[Parallel(n_jobs=-1)]: Done 12632 tasks      | elapsed:   59.4s
[Parallel(n_jobs=-1)]: Done 16032 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 19832 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 24032 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 28632 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 33632 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 39032 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 44832 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 51032 tasks      | el

Time elapsed: 0:06:07.378326
-0.1609113678059338
MSE : 99.935141%, 
In Micronesia


/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:298: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:303: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [9]:
def Resmaking_for_contr(df, country):
    tmp = list(makexy(df, country))
    tmp.append(tmp[0].columns)
        
    X_train, X_test, y_train, y_test = train_test_split(tmp[0].as_matrix(), tmp[1].as_matrix(), test_size=0.1, random_state=42)
    p = Testing_grid_xgb(X_train, y_train)
        
    model = xgb.XGBRegressor(max_depth=p['max_depth'], learning_rate = p['learning_rate'], 
                                 n_estimators = p['n_estimators'], objective = p['objective'], 
                                 gamma = p['gamma'], min_child_weight = p['min_child_weight'], 
                                 max_delta_step = p['max_delta_step'], subsample = p['subsample'], 
                                 colsample_bytree = p['colsample_bytree'], reg_alpha = p['reg_alpha'], 
                                 reg_lambda = p['reg_lambda'], scale_pos_weight = p['scale_pos_weight'], 
                                 base_score = p['base_score'])
    model.fit(X_train, y_train)
        
    print ('MSE : {0:f}%, \nIn {1:s}'.format(100-mean_squared_error(model.predict(X_test), y_test), country))
        
    model.fit(tmp[0], tmp[1])
    tmp[0] = np.nan_to_num(tmp[0].as_matrix())
    tmp[1] = np.nan_to_num(tmp[1].as_matrix())
    
    rez = pd.DataFrame(model.feature_importances_, index = tmp[2]).sort_values(by = [0], ascending=False)
    rez[rez[0]!=0].to_excel('Importance/Varible_reg_in_'+str(country)+'.xlsx')
        
    rez = pd.concat([pd.DataFrame(f_regression(tmp[0], tmp[1].ravel())[0], columns=['F-value'], index=tmp[2]),
                     pd.DataFrame(f_regression(tmp[0], tmp[1].ravel())[1], columns=['p-value'], index=tmp[2])], axis=1)
    rez['p-value'] = rez['p-value'].apply(lambda x: round(x, 5))
        
    rez[rez['p-value']<0.01].sort_values(by = ['F-value'], ascending=False).to_excel('Importance/Varible_f_reg_in_'+str(country)+'.xlsx')
        
    rez = pd.DataFrame(mutual_info_regression(tmp[0], tmp[1].ravel()),
                           index = tmp[2]).sort_values(by = [0], ascending=False)
    rez[rez[0]!=0].to_excel('Importance/Varible_muatal_reg_in_'+str(country)+'.xlsx')

In [10]:
Resmaking_for_contr(df, 'RUS')

Fitting 8 folds for each of 10000 candidates, totalling 80000 fits


[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 1240 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 3240 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 6040 tasks      | elapsed:   29.6s
[Parallel(n_jobs=-1)]: Done 9640 tasks      | elapsed:   46.4s
[Parallel(n_jobs=-1)]: Done 14040 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 19240 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 25240 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 32040 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 39640 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 48040 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 57240 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 67240 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 78040 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 80000 out of 80000 | elapsed:  6.3min finished


Time elapsed: 0:06:21.570331
-0.36721313107861697
MSE : 97.812259%, 
In RUS


/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:298: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/home/denis/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:303: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/denis/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
